In [26]:
import tensorflow_decision_forests as tfdf
from tensorflow import metrics
import os
import sys
import numpy as np
import datatable as dt
from datatable import f
import pandas as pd
import tensorflow as tf
import math
import random
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'umap'

In [2]:
data = pd.read_csv("../data/processed/merged_reduced.csv", index_col=0)

In [3]:
data_reduced = data.drop(data.columns[np.random.choice(range(1, data.shape[1]-1), size=9000, replace=False)], axis=1)
data_reduced = data_reduced.drop(np.random.choice(range(1, data_reduced.shape[0]), size=3000, replace=False), axis=0)

In [4]:
data_reduced

,C10,AL627309.1,AL627309.4,LINC01409,HES4,TTLL10-AS1,B3GALT6,AL162741.1,AL645728.1,CDK11B,...,FAM41AY2,AC022486.1,AC007244.1,TTTY6B,RBMY1J,MT-CO2,MT-ND4L,AC011841.1,AL354822.1,cell_type
0,AAACAGCCAAAGCCTC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,80.0,6.0,0.0,0.0,singlet
6,AAACCAACATTGTCCT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.0,3.0,0.0,0.0,singlet
7,AAACCGAAGCGTGCGT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0,singlet
9,AAACCGCGTCTAACCT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,121.0,4.0,0.0,0.0,homo
13,AAACCGGCAAAGCTCC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,2.0,0.0,0.0,singlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,TTTGCGACAGATAGAC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0,singlet
3993,TTTGGCTGTGACATAT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,50.0,4.0,0.0,0.0,singlet
3996,TTTGGTAAGTAACGGA.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,168.0,8.0,0.0,0.0,singlet
4001,TTTGTCTAGCTGTACG.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,singlet


## Feature selection -> reduce number of columns
- filter correlated columns

## Try out Projected data
- UMAP ?
- t-SNE
- PCA

### feature selection

In [10]:
features = data_reduced.iloc[:, 1:-1]

In [11]:
corr_mat = features.corr().abs()

In [12]:
upper_tri = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))

/tmp/ipykernel_681515/4277379874.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))


In [13]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
len(to_drop)

68

In [24]:
features_filtered = features.drop(to_drop, axis=1)

In [25]:
features_filtered

,AL627309.1,AL627309.4,LINC01409,HES4,TTLL10-AS1,B3GALT6,AL162741.1,AL645728.1,CDK11B,AL139246.5,...,AC011751.1,FAM41AY2,AC022486.1,AC007244.1,TTTY6B,RBMY1J,MT-CO2,MT-ND4L,AC011841.1,AL354822.1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,80.0,6.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17.0,3.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,121.0,4.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0
3993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,4.0,0.0,0.0
3996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,168.0,8.0,0.0,0.0
4001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0


### Project data